<a href="https://colab.research.google.com/github/ananya0-0/Hand-Gesture-Recognition-/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GITHUB REPO SETUP


In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ananya365","key":"30292cd59f8111026a3649b4006c0b95"}'}

In [ ]:
!pip install streamlit opencv-python-headless scikit-learn
!npm install localtunnel
!npm audit fix --force

UPLOADING KAGGLE API TOKEN

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d grassknoted/asl-alphabet

!unzip -o -q asl-alphabet.zip -d asl_alphabet_train

print("Dataset unzipped successfully!")

REQUIRED ML PIPELINES AND DATA TRAINING


In [ ]:
!pip uninstall -y mediapipe protobuf
!pip install mediapipe==0.10.14 protobuf==4.25.3 # mediapipe 0.10.14 still has 'solutions'

In [ ]:
import os
import cv2
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score


#finding data
def find_dataset_root(start_dir='/content'):
    print("Scanning for dataset...")
    for root, dirs, files in os.walk(start_dir):
        # We look for a folder named 'A' that contains images
        if 'A' in dirs:
            path_to_a = os.path.join(root, 'A')
            # Check if 'A' actually has images inside
            if any(f.endswith(('.jpg', '.png')) for f in os.listdir(path_to_a)):
                print(f"FOUND IT! Dataset is located at: {root}")
                return root
    return None

# Locate the dataset automatically
dataset_dir = find_dataset_root()

if dataset_dir is None:
    print("\nCRITICAL FAILURE: Could not find the folder 'A' anywhere.")
    print("Please manually check if you unzipped the file 'asl-alphabet.zip'.")
    print("Run: !unzip asl-alphabet.zip")
    exit()

# Setup paths
model_save_dir = '/content/ASL_Model'
os.makedirs(model_save_dir, exist_ok=True)
model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
label_map_path = os.path.join(model_save_dir, 'label_map.pkl')

# Feature extraction function
def extract_features(img_path, hands_model):
    img = cv2.imread(img_path)
    if img is None: return None
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands_model.process(img_rgb)
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmark_vector = []
        wrist_x = hand_landmarks.landmark[0].x
        wrist_y = hand_landmarks.landmark[0].y
        for landmark in hand_landmarks.landmark:
            landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])
        return np.array(landmark_vector)
    return None

def train_model():
    features = []
    labels = []
    label_map = {}
    current_label_id = 0

    # Initialize MediaPipe
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

    # Get all class folders (A, B, C...)
    all_class_dirs = sorted([d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))])

    print(f"Processing {len(all_class_dirs)} classes...")

    for class_name in all_class_dirs:
        # Skip weird folders not part of the alphabet
        if class_name in ['nothing', 'space', 'del'] or len(class_name) > 1:
             # You can remove this 'if' to include them, but let's stick to A-Z for safety first
             pass

        if class_name not in label_map:
            label_map[class_name] = current_label_id
            current_label_id += 1

        class_path = os.path.join(dataset_dir, class_name)
        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        # Training on first 30 images per class to ensure it works fast
        # You can increase this number later
        for image_file in image_files[:30]:
            feature_vector = extract_features(os.path.join(class_path, image_file), hands)
            if feature_vector is not None:
                features.append(feature_vector)
                labels.append(label_map[class_name])

    hands.close()

    if len(features) == 0:
        print("Error: No features extracted. Something is wrong with the images.")
        return

    print(f"✅ Extracted features from {len(features)} images.")

    # Train
    X = np.array(features)
    y = np.array(labels)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500)
    mlp.fit(X_train, y_train)

    # Save
    with open(model_path, 'wb') as f: pickle.dump(mlp, f)
    with open(label_map_path, 'wb') as f: pickle.dump({v: k for k, v in label_map.items()}, f)

    print(f"\nSUCCESS! Model saved to: {model_path}")
    print("You can now run your Streamlit app.")

if __name__ == "__main__":
    train_model()

In [ ]:
!python train_model.py

python3: can't open file '/content/train_model.py': [Errno 2] No such file or directory


In [ ]:
import os
if os.path.exists("/content/ASL_Model/asl_classifier.pkl"):
    print("File found! You can now launch Streamlit.")
else:
    print("File still missing. Check the output of Step 1 for errors.")

In [ ]:
!git add train_model.py
!git commit -m "adding full training pipeline"

CREATING THE STREAMLIT APP


In [ ]:
!pip install streamlit-webrtc av

In [ ]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
import pickle
import mediapipe as mp
import av
# --- FIX: Import WebRtcMode directly ---
from streamlit_webrtc import webrtc_streamer, VideoTransformerBase, RTCConfiguration, WebRtcMode

# --- CONFIG TO FIX CONNECTION ISSUES ---
RTC_CONFIGURATION = RTCConfiguration(
    {"iceServers": [{"urls": ["stun:stun.l.google.com:19302"]}]}
)

# --- LOAD MODEL ---
model_path = '/content/ASL_Model/asl_classifier.pkl'
label_map_path = '/content/ASL_Model/label_map.pkl'

try:
    with open(model_path, 'rb') as f: model = pickle.load(f)
    with open(label_map_path, 'rb') as f: label_map = pickle.load(f)
except FileNotFoundError:
    st.error("Model not found!")
    st.stop()

# --- MEDIAPIPE SETUP ---
try:
    mp_hands = mp.solutions.hands
except AttributeError:
    from mediapipe.python.solutions import hands as mp_hands

# --- REAL-TIME VIDEO PROCESSOR ---
class VideoProcessor(VideoTransformerBase):
    def __init__(self):
        self.hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

    def recv(self, frame):
        img = frame.to_ndarray(format="bgr24")

        # Convert to RGB
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = self.hands.process(img_rgb)

        prediction_text = "Show Hand"
        box_color = (0, 0, 255)

        if results.multi_hand_landmarks:
            box_color = (0, 255, 0)
            for hand_landmarks in results.multi_hand_landmarks:
                mp.solutions.drawing_utils.draw_landmarks(
                    img, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                wrist_x = hand_landmarks.landmark[0].x
                wrist_y = hand_landmarks.landmark[0].y
                landmark_vector = []
                for landmark in hand_landmarks.landmark:
                    landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])

                try:
                    prediction = model.predict([landmark_vector])
                    prediction_text = label_map.get(prediction[0], "Unknown")
                except Exception:
                    prediction_text = "Error"

        cv2.rectangle(img, (0,0), (640, 60), (0,0,0), -1)
        cv2.putText(img, f"Prediction: {prediction_text}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, box_color, 2, cv2.LINE_AA)

        return av.VideoFrame.from_ndarray(img, format="bgr24")

# --- UI LAYOUT ---
st.title("🖐️ Real-Time ASL Recognition")
st.write("Click 'Start' below. If it asks for permissions, click Allow.")

webrtc_streamer(
    key="asl-detection",
    mode=WebRtcMode.SENDRECV, # <--- FIXED HERE
    rtc_configuration=RTC_CONFIGURATION,
    video_processor_factory=VideoProcessor,
    media_stream_constraints={"video": True, "audio": False},
    async_processing=True,
)

In [ ]:
!curl ipv4.icanhazip.com

In [ ]:
#launching the app
print("Your app is running! Click the link below ending in .loca.lt")
!streamlit run app.py & npx localtunnel --port 8501